In [10]:
import time
import pandas as pd
from multiprocessing import Pool
import math
import os
import shutil
import matplotlib.pyplot as plt
from scipy.io import readsav
import glob
import statistics
from aetherpy.io import read_routines
from math import cos, radians, sin, sqrt
from scipy import spatial, signal
from glob import glob
from datetime import datetime
from datetime import timedelta
from struct import unpack
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from pylab import cm
import fnmatch

from aetherpy.io import read_routines as rr

import sys

from scipy.interpolate import LinearNDInterpolator, interp1d

from utility_programs.read_routines import SAMI
from utility_programs.filters import make_fits

import xarray as xr
import importlib
import gc

import cartopy.crs as ccrs

import xesmf as xe
import esmpy

In [34]:
import ESMF

ModuleNotFoundError: No module named 'ESMF'

In [35]:
from ESMF.util.cache_data import cache_data_file
import os
DD = os.path.join(os.getcwd(), "examples/data")
if not os.path.isdir(DD):
    os.makedirs(DD)
cache_data_file(os.path.join(DD, "ll1deg_grid.nc"))

ModuleNotFoundError: No module named 'ESMF'

In [4]:
sami_data_path = "/petastore/phil/GITM/cheyenne_runs/FullAmp/sami-gitm-coupled/"

In [5]:

sami_og_vars = {
    'deneu.dat': 'edens', }

geo_grid_files = {
    'lat': 'glatu.dat', 'lon': 'glonu.dat', 'alt': 'zaltu.dat',
}

In [6]:
nz, nf, nlt, nt = SAMI.get_grid_elems_from_parammod(sami_data_path)
times = SAMI.make_times(nt, sami_data_path, datetime(2011, 5, 20))

In [7]:
grid = {}

for f in geo_grid_files:
    file = open(os.path.join(sami_data_path, geo_grid_files[f]), 'rb')
    raw = np.fromfile(file, dtype='float32')[1:-1].copy()
    file.close()

    grid[f] = raw.reshape(nlt, nf, nz).copy()

In [8]:
ds_in = xr.Dataset(coords={
    "Latitude": (['loc'], grid['lat'].flatten(), {'units': 'degrees_north'}),
    "Longitude": (["loc"], grid['lon'].flatten(), {'units': 'degrees_east'}),
    "Height": (["loc"], grid['alt'].flatten(), {'units': "kilometers"})
}
)

In [9]:
ds_in

<xarray.Dataset>
Dimensions:    (loc: 1474560)
Coordinates:
    Latitude   (loc) float32 -13.74 -13.74 -13.74 -13.74 ... 80.04 80.04 80.04
    Longitude  (loc) float32 290.3 290.3 290.3 290.3 ... 277.6 277.6 277.6 277.6
    Height     (loc) float32 80.02 80.03 80.05 80.06 ... 92.96 89.94 87.09 84.4
Dimensions without coordinates: loc
Data variables:
    *empty*

In [14]:
locs = esmpy.LocStream(len(ds_in.Latitude), coord_sys=esmpy.CoordSys.SPH_DEG)

In [17]:
locs['ESMF:Lat'] = ds_in.Latitude.values
locs['ESMF:Lon'] = ds_in.Longitude.values
locs['ESMF:Height'] = ds_in.Height.values

In [18]:
field = esmpy.Field(locs, name='field', typekind=esmpy.TypeKind.R4,
                    staggerloc=esmpy.StaggerLoc.CENTER_VCENTER)

Field:
    name = 'field'
    type = <TypeKind.R4: 5>
    rank = 1
    extra dimensions = 0
    staggerloc = <StaggerLoc.CENTER: 0>
    lower bounds = array([0], dtype=int32)
    upper bounds = array([1474560], dtype=int32)
    extra bounds = None
    data = array([8.360447e+35, 1.548155e-41, 8.360701e+35, ..., 0.000000e+00,
       0.000000e+00, 0.000000e+00], dtype=float32)
    grid = 
LocStream:
    name = None 
    lower_bounds = array([0], dtype=int32) 
    upper_bounds = array([1474560], dtype=int32) 
    keys = dict_items([('ESMF:Lat', array([-13.743711, -13.742006, -13.740195, ...,  80.03691 ,  80.0375  ,
        80.0378  ], dtype=float32)), ('ESMF:Lon', array([290.29834, 290.29816, 290.29797, ..., 277.58685, 277.58087,
       277.57538], dtype=float32)), ('ESMF:Height', array([80.01807, 80.03076, 80.04541, ..., 89.93799, 87.08545, 84.40137],
      dtype=float32))]) 

)

In [25]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, 2.0), {'units': 'degrees_north'}),
        "lon": (["lon"], np.arange(0, 360, 5), {'units': 'degrees_east'}),
        "height": (["elevation"], np.arange(250, 2000, 50), {'units': "kilometers"})
    }
)

In [26]:
ds_out.sizes

Frozen({'lat': 90, 'lon': 72, 'elevation': 35})

In [32]:
dst = esmpy.Grid(np.array([90, 72, 35]), coord_sys=esmpy.CoordSys.SPH_DEG)

gridXCenter = dst.get_coords(0)
gridXCenter[...] = ds_out.lat.values

gridYCenter = dst.get_coords(1)
gridYCenter[...] = ds_out.lon.values

gridZCenter = dst.get_coords(2)
gridZCenter[...] = ds_out.height.values

AssertionError: 

In [33]:
dst.get_coords

<bound method Grid.get_coords of Grid:
    type = <TypeKind.R8: 6>    areatype = <TypeKind.R8: 6>    rank = 3    num_peri_dims = 0    periodic_dim = None    pole_dim = None    pole_kind = None    coord_sys = <CoordSys.SPH_DEG: 1>    staggerloc = [False, False, False, False, False, False, False, False]    lower bounds = [None, None, None, None, None, None, None, None]    upper bounds = [None, None, None, None, None, None, None, None]    coords = [[None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None]]    mask = [None, None, None, None, None, None, None, None]    area = [None, None, None, None, None, None, None, None]>

In [64]:
ds_in = ds_in.where((ds_in.height > 400) & (ds_in.height < 600), drop=True)
ds_in

<xarray.Dataset>
Dimensions:  (loc: 113520)
Coordinates:
    lat      (loc) float32 -15.29 -15.03 -14.78 -14.54 ... 79.96 79.96 79.96
    lon      (loc) float32 290.4 290.4 290.4 290.4 ... 278.3 278.3 278.2 278.2
    height   (loc) float32 400.0 403.5 406.7 409.6 ... 457.2 441.3 425.9 411.0
Dimensions without coordinates: loc
Data variables:
    *empty*

In [65]:
ds_in.to_netcdf('in.nc')

In [66]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, 2.0), {'units': 'degrees_north'}),
        "lon": (["lon"], np.arange(0, 360, 5), {'units': 'degrees_east'}),
        "height": (["elevation"], np.arange(450, 600, 50), {'units': "kilometers"})
    }
)

In [67]:
ds_out

<xarray.Dataset>
Dimensions:  (lat: 90, lon: 72, elevation: 3)
Coordinates:
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0
  * lon      (lon) int64 0 5 10 15 20 25 30 35 ... 325 330 335 340 345 350 355
Dimensions without coordinates: elevation
Data variables:
    height   (elevation) int64 450 500 550

In [68]:
ds_out.to_netcdf('out.nc')

In [1]:
import esmpy

In [2]:
ds_in

NameError: name 'ds_in' is not defined

In [ ]:
esmpy.LocStream(esmpy.LocStreamFromFile, 'in.nc', 'out.nc')